In [89]:
import pandas as pd 
import numpy as np
import hashlib
pd.set_option('display.max_colwidth', None)

In [130]:
def hash_str(to_hash): 
    stringToHash = to_hash.encode()
    return hashlib.sha256(stringToHash).hexdigest()

In [131]:
def process_lbl(line_list): 
    name_dict = {}
    data_dict = {}
    name =''
    i = 0
    while (i < len(line_list)): 
        if i+1 < len(line_list) and line_list[i+1] == '---------,---------,---------,---------\n':
            name = hash_str(line_list[i])
            name_dict[name] = line_list[i]
            data_dict[name] = []
            i = i + 2
        else: 
            if line_list[i] != 'Component,test,subtest,metric,metric_type\n':
                if "Final Checks, Consistency Verification" in line_list[i]: 
                    line_list[i] = line_list[i].replace("Final Checks, Consistency Verification","Final Checks Consistency Verification")
                data_dict[name].append((name+','+line_list[i].strip()).split(','))
            i = i + 1
    return (name_dict, data_dict)
            
        
        

In [132]:
f = open("tests/results/pihole.txt", "r")
lbl = [x for x in f]
f.close()

name_dict, data_dict = process_lbl(lbl)

In [136]:
df = pd.DataFrame()
for k in data_dict:
    temp_df = pd.DataFrame(data_dict[k])
    df = df.append(temp_df,ignore_index = True)
df.columns = ['name','type','test','sub_test','val','metric']

In [137]:
df.head()

,name,type,test,sub_test,val,metric
0,10a43b1f25dff78109b509110e86bc7718f48aeec86dc73926bebb9a27eaa894,compiler,Compiler,Full,9361946,μs
1,10a43b1f25dff78109b509110e86bc7718f48aeec86dc73926bebb9a27eaa894,verifier,Verification,Full,2187137,μs
2,10a43b1f25dff78109b509110e86bc7718f48aeec86dc73926bebb9a27eaa894,solver,SAFA Solving,Clear Match,1,μs
3,10a43b1f25dff78109b509110e86bc7718f48aeec86dc73926bebb9a27eaa894,compiler,R1CS,Proof Setup,8259604,μs
4,10a43b1f25dff78109b509110e86bc7718f48aeec86dc73926bebb9a27eaa894,prover,prove,prove_0,279927,μs


In [121]:
df.sub_test.unique()

array(['Full', 'Clear Match', 'Proof Setup', 'prove_0',
       'Nova Verification', 'R1CS precomputations', '"Final Checks',
       'To Circuit', 'step_0', 'SAFA', 'Framework Solve',
       'Commitment Generations', 'Consistency Proofs', 'Compressed SNARK',
       'Primary Circuit', 'Secondary Circuit', 'Compressed SNARK size'],
      dtype=object)

In [122]:
df.loc[(df.metric == 'μs')].groupby(['name','type']).val.sum()

name                                                              type    
10a43b1f25dff78109b509110e86bc7718f48aeec86dc73926bebb9a27eaa894  compiler     9361946825960464270621563388960236
                                                                  prover                      2799272240162412155
                                                                  solver                              11146319751
                                                                  verifier                         21871372026875
41c3bd6aa3af29157e398ed29f2ce37853ab614986be0d8b6b767c52a94cc267  compiler      860158234756484811365147412194635
                                                                  prover                      1906932669922381915
                                                                  solver                                513796220
                                                                  verifier                         20017602156130
4511538acb6ea

In [123]:
df.val.astype(int)

ValueError: invalid literal for int() with base 10: ' Consistency Verification"'

In [124]:
df.loc[df.name=='10a43b1f25dff78109b509110e86bc7718f48aeec86dc73926bebb9a27eaa894']

,name,type,test,sub_test,val,metric
0,10a43b1f25dff78109b509110e86bc7718f48aeec86dc73926bebb9a27eaa894,compiler,Compiler,Full,9361946,μs
1,10a43b1f25dff78109b509110e86bc7718f48aeec86dc73926bebb9a27eaa894,verifier,Verification,Full,2187137,μs
2,10a43b1f25dff78109b509110e86bc7718f48aeec86dc73926bebb9a27eaa894,solver,SAFA Solving,Clear Match,1,μs
3,10a43b1f25dff78109b509110e86bc7718f48aeec86dc73926bebb9a27eaa894,compiler,R1CS,Proof Setup,8259604,μs
4,10a43b1f25dff78109b509110e86bc7718f48aeec86dc73926bebb9a27eaa894,prover,prove,prove_0,279927,μs
5,10a43b1f25dff78109b509110e86bc7718f48aeec86dc73926bebb9a27eaa894,verifier,Verification,Nova Verification,2026875,μs
6,10a43b1f25dff78109b509110e86bc7718f48aeec86dc73926bebb9a27eaa894,compiler,R1CS,R1CS precomputations,64270,μs
7,10a43b1f25dff78109b509110e86bc7718f48aeec86dc73926bebb9a27eaa894,verifier,Verification,"""Final Checks","Consistency Verification""",159262
8,10a43b1f25dff78109b509110e86bc7718f48aeec86dc73926bebb9a27eaa894,compiler,R1CS,To Circuit,62156,μs
9,10a43b1f25dff78109b509110e86bc7718f48aeec86dc73926bebb9a27eaa894,solver,Witness Generation,step_0,11463,μs
